In [0]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
 
CREATE OR ALTER PROCEDURE [CFG].[usp_UpdateDataSourceCDCPipeline]
    @InternalProductID      VARCHAR(5),
    @InternalClientId       VARCHAR(10) = NULL, -- Optional
    @InternalFacilityId     VARCHAR(10) = NULL, -- Optional
    @TableToUpdate          VARCHAR(100),
    @GatewayPipelineID      VARCHAR(100),
    @IngestionPipelineID    VARCHAR(100),
    @CatalogDatabaseName    VARCHAR(1000),
    @SourceServerName       VARCHAR(1000) = NULL,  -- Optional
    @SourceDatabaseName     VARCHAR(100) = NULL    -- Optional
AS
BEGIN
    DECLARE @SQL NVARCHAR(MAX);
 
    IF @TableToUpdate = 'ProductDataSourceInstance'
        SET @SQL = 'UPDATE cfg.ProductDataSourceInstance' +
               ' SET CDCgatewayPipelineID = ''' + @GatewayPipelineID + ''', ' +
               'CDCingestionPipelineID = ''' + @IngestionPipelineID + ''', ' +
               'CDCDestinationSchema = ''' + @CatalogDatabaseName + ''' ' +
               'WHERE Active = 1 AND InternalProductId = ' + @InternalProductID +
               ' AND SourceServerName1 = ''' + @SourceServerName + ''' ' +
               ' AND SourceDatabaseName1 = ''' + @SourceDatabaseName + ''''
 
    ELSE IF @TableToUpdate = 'ProductDataSource'
        SET @SQL = 'UPDATE cfg.ProductDataSource' +
               ' SET CDCgatewayPipelineID = ''' + @GatewayPipelineID + ''', ' +
               'CDCingestionPipelineID = ''' + @IngestionPipelineID + ''', ' +
               'CDCDestinationSchema = ''' + @CatalogDatabaseName + ''' ' +
               'WHERE InternalProductId = ' + @InternalProductID +
               ' AND SourceDatabaseName1 = ''' + @SourceDatabaseName + '''' +
               ' AND SourceServerName1 = ''' + @SourceServerName + ''''
 
    ELSE
    BEGIN
        SET @SQL = 'UPDATE cfg.' + QUOTENAME(@TableToUpdate) + -- ProductClient or ProductClientFacility
               ' SET CDCgatewayPipelineID = ''' + @GatewayPipelineID + ''', ' +
               'CDCingestionPipelineID = ''' + @IngestionPipelineID + ''', ' +
               'CDCDestinationSchema = ''' + @CatalogDatabaseName + ''' ' +
               'WHERE Active = 1 AND InternalProductId = ' + @InternalProductID + ' '
 
        IF @InternalClientId IS NOT NULL
            SET @SQL = @SQL + 'AND InternalClientId = ' + @InternalClientId + ' '
        ELSE
            SET @SQL = @SQL + 'AND InternalClientId = 0 '
 
        IF @InternalFacilityId IS NOT NULL
            SET @SQL = @SQL + 'AND InternalFacilityId = ' + @InternalFacilityId + ' '
        ELSE
            IF @TableToUpdate = 'ProductDataSourceClientFacility'
                SET @SQL = @SQL + 'AND InternalFacilityId = 0 '
    END
 
    EXECUTE(@SQL);
 
    SELECT 1 AS status;
END
GO